In [1]:
from flickrapi import FlickrAPI
import pandas as pd
import os



In [2]:
key = '0e4e7a8ceb3f350abd0a743812365c51'
secret = '3172a3f6ec67c50f'
sizes = ['url_c', 'url_z', 'url_m', 'url_n', 'url_s', 'url_t']

In [3]:
def get_photos(image_tag):
    extras = ','.join(sizes)
    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,  # it will search by image title and image tags
                            extras=extras,  # get the urls for each size we want
                            privacy_filter=1,  # search only for public photos
                            per_page=50,
                            sort='relevance')  # we want what we are looking for to appear first
    return photos

In [4]:
get_photos('tortilla de patata')

<generator object FlickrAPI.data_walker at 0x7f2111204ed0>

In [161]:
def get_url(photo):
    for i in range(len(sizes)):  # makes sure the loop is done in the order we want
        url = photo.get(sizes[i])
        #print('Encontrado el tamaño ',sizes[i])
        if url:  # if url is None try with the next size
            print('Encontrado el tamaño ',sizes[i])
            return url

In [171]:
def get_urls(image_tag, folder, max):
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            #counter += 1
            #print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                print(url)
                print(url.split("/")[-1])
                if os.path.isfile(os.path.join('/home/dsc/tfm_data/', folder, url.split("/")[-1])):
                    print('El archivo existe.')
                        #continue
                else:
                    print('El archivo no existe')
                    urls.append(url)
                    counter += 1
            except:    
                 print("Url for image number {} could not be fetched".format(count))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    # En este punto sería leer el csv (si existe) y añadir las nuevas urls
    #urls = pd.Series(urls)
    urls = pd.DataFrame(urls, index=None)
    print(urls.head())
    urls.to_csv(image_tag + '_urls.csv')
    print('Listo!')
    

    return urls

In [178]:
def get_urls(image_tag, max):
    
    # Fijamos el cwd en el que trabajamos:
    os.chdir('/home/dsc/TFM_food_recognition/image_urls')
    
    # Comprobamos si anteriormente hemos descargado imágenes con el tag del param.
    if os.path.isfile(os.path.join('/home/dsc/TFM_food_recognition/image_urls', image_tag + '_urls.csv')):
        print('Descargado anteriormente')
        urls_book = pd.read_csv(image_tag + '_urls.csv')
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            #counter += 1
            #print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                print(url)
                print(url.split("/")[-1])
                
                # Si tenemos un histŕotico de las urls comprobamos si la hemos descargado:
                if urls_book is not None:
                    url_patron = url.split("/")[-1]
                    print(url_patron)
                    
                    if urls_book['urls'].str.contains(url_patron).any():
                        print('url descargada anteriormente')
                    else:
                        print('url no descargada anteriormente')
                        urls.append(url)
                        counter += 1
                # Si no lo tenemos, todas las urls son nuevas:
                else:
                    print('url descargada por primera vez')
                    urls.append(url)
                    counter += 1
            except:    
                 print("Url for image number {} could not be fetched".format(counter))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    # En este punto sería leer el csv (si existe) y añadir las nuevas urls
    #urls = pd.Series(urls)
    urls = pd.DataFrame(urls)
    urls.columns = ['urls']
    urls['download']=False
    print(urls.head())
    
    if urls_book is not None:
        urls_book.append(urls)
        urls_book.to_csv(image_tag + '_urls.csv', index=None)
    else:
        urls.to_csv(image_tag + '_urls.csv', index=None)
        print('Listo!')
    

    return urls

In [203]:
def get_urls(image_tag, max):
    
    # Fijamos el cwd en el que trabajamos:
    os.chdir('/home/dsc/TFM_food_recognition/image_urls')
    
    # Comprobamos si anteriormente hemos descargado imágenes con el tag del param.
    if os.path.isfile(os.path.join('/home/dsc/TFM_food_recognition/image_urls', image_tag + '_urls.csv')):
        print('Archivo de urls descargado anteriormente')
        urls_book = pd.read_csv(image_tag + '_urls.csv')
        
    # Variable que controla la existencia previa del archivo con urls del image_tag:
    try:
        urls_book
    except NameError:
        urls_book_is = False
    else:
        urls_book_is = True
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            #counter += 1
            #print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                print(url)
                print(url.split("/")[-1])
                
                # Si tenemos un histŕotico de las urls comprobamos si la hemos descargado:
                if urls_book_is: 
                    url_patron = url.split("/")[-1]
                    print(url_patron)
                    
                    if urls_book['urls'].str.contains(url_patron).any():
                        print('url descargada anteriormente')
                    else:
                        print('url no descargada anteriormente')
                        urls.append(url)
                        counter += 1
                # Si no lo tenemos, todas las urls son nuevas:
                else:
                    print('url descargada por primera vez')
                    urls.append(url)
                    counter += 1
            except:    
                 print("Url for image number {} could not be fetched".format(counter))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    # En este punto sería leer el csv (si existe) y añadir las nuevas urls
    #urls = pd.Series(urls)
    urls = pd.DataFrame(urls)
    urls.columns = ['urls']
    urls['download']=False
    print(urls.head())
    
    if urls_book_is:
        print('uniendo dfs')
        urls_book = urls_book.append(urls)
        print(len(urls_book))
        urls_book.to_csv(image_tag + '_urls.csv', index=None)
    else:
        urls.to_csv(image_tag + '_urls.csv', index=None)
        print('Listo!')
    

    return urls

In [201]:
try:
    urls_book
except NameError:
    urls_book_is = False
else:
    urls_book_is = True
urls_book_is

False

In [202]:
who


FlickrAPI	 counter	 download_images	 get_photos	 get_url	 get_urls	 image_tag	 key	 max	 
os	 patron	 pd	 photo	 photos	 pru	 pru2	 prueba	 prueba_leido	 
requests	 secret	 sizes	 url_buscar	 urls	 urls_book_is	 urls_exists	 


In [323]:
prueba = get_urls(image_tag = 'cocido', max = 10) # 10 fotos nuevas que no tengamos

Archivo de urls descargado anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/8077/8428940257_15f6a02bfd_c.jpg
8428940257_15f6a02bfd_c.jpg
8428940257_15f6a02bfd_c.jpg
url descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/8658/15752325014_612de967d8_c.jpg
15752325014_612de967d8_c.jpg
15752325014_612de967d8_c.jpg
url descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/3357/3333547544_814640b120_c.jpg
3333547544_814640b120_c.jpg
3333547544_814640b120_c.jpg
url descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/8622/16348796416_771690f621_c.jpg
16348796416_771690f621_c.jpg
16348796416_771690f621_c.jpg
url descargada anteriormente
Encontrado el tamaño  url_z
https://live.staticflickr.com/8/10560929_d57c09863b_z.jpg
10560929_d57c09863b_z.jpg
10560929_d57c09863b_z.jpg
url descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/6/10560925_3660939718_c.

Encontrado el tamaño  url_c
https://live.staticflickr.com/6118/6345534586_e16b5a4694_c.jpg
6345534586_e16b5a4694_c.jpg
6345534586_e16b5a4694_c.jpg
url no descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/1212/1428634372_632f894e4c_c.jpg
1428634372_632f894e4c_c.jpg
1428634372_632f894e4c_c.jpg
url no descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/4087/5190324145_d9c7fa5446_c.jpg
5190324145_d9c7fa5446_c.jpg
5190324145_d9c7fa5446_c.jpg
url no descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/152/433692896_2eb3fe7c77_c.jpg
433692896_2eb3fe7c77_c.jpg
433692896_2eb3fe7c77_c.jpg
url no descargada anteriormente
Encontrado el tamaño  url_c
https://live.staticflickr.com/5140/5455330903_d2b0f63568_c.jpg
5455330903_d2b0f63568_c.jpg
5455330903_d2b0f63568_c.jpg
url no descargada anteriormente
Encontrado el tamaño  url_z
https://live.staticflickr.com/7521/16080327106_73e73d592d_z.jpg
16080327106_73e73

In [321]:
import requests
import os

def download_images(urls_file, folder, ruta_destino):
    os.chdir("/home/dsc/TFM_food_recognition/image_urls")
    
    urls = []
    doc = pd.read_csv(urls_file)
    new_urls = doc[doc['download'] == False]
    
    for index, row in new_urls.iterrows():
        if row['urls'].startswith('https'):
            urls.append(row['urls'])
    
    if not os.path.isdir(os.path.join(ruta_destino, folder.replace(" ", "_"))):
        os.mkdir(os.path.join(ruta_destino, folder.replace(" ", "_")))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join(ruta_destino, folder.replace(" ", "_"),url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))
    
    doc.loc[doc['download']==False, 'download'] = True
    doc.to_csv(urls_file, index=None)
    

In [324]:
download_images(urls_file="cocido_urls.csv", folder= 'cocido madrilenio', ruta_destino='/home/dsc/tfm_data')

Done downloading 1 of 10
Done downloading 2 of 10
Done downloading 3 of 10
Done downloading 4 of 10
Done downloading 5 of 10
Done downloading 6 of 10
Done downloading 7 of 10
Done downloading 8 of 10
Done downloading 9 of 10
Done downloading 10 of 10


In [288]:
os.chdir("/home/dsc/TFM_food_recognition/image_urls")
file_name = 'cocido_urls.csv'
doc = pd.read_csv(file_name)  
doc.head(5)

,urls,download
0,https://live.staticflickr.com/8077/8428940257_...,False
1,https://live.staticflickr.com/8658/15752325014...,False
2,https://live.staticflickr.com/3357/3333547544_...,False
3,https://live.staticflickr.com/8622/16348796416...,False
4,https://live.staticflickr.com/8/10560929_d57c0...,False


In [266]:
type(doc)

pandas.core.frame.DataFrame

In [267]:
doc.urls[0].startswith('https')

True

In [244]:
urls = []
for index, row in doc.iterrows():
    if row['urls'].startswith('https'):
        urls.append(row['urls'])
urls        

['https://live.staticflickr.com/8077/8428940257_15f6a02bfd_c.jpg',
 'https://live.staticflickr.com/8658/15752325014_612de967d8_c.jpg',
 'https://live.staticflickr.com/3357/3333547544_814640b120_c.jpg',
 'https://live.staticflickr.com/8622/16348796416_771690f621_c.jpg',
 'https://live.staticflickr.com/8/10560929_d57c09863b_z.jpg',
 'https://live.staticflickr.com/6/10560925_3660939718_c.jpg',
 'https://live.staticflickr.com/5251/5398039808_4a34434460_c.jpg',
 'https://live.staticflickr.com/6/10560928_0ebeb97562_z.jpg',
 'https://live.staticflickr.com/127/321323177_3e057a4ef6_c.jpg',
 'https://live.staticflickr.com/8785/18276105075_edef70fc09_c.jpg',
 'https://live.staticflickr.com/5/10560927_e6a8465f45.jpg',
 'https://live.staticflickr.com/4/4456745_cebf9bb9c5_n.jpg',
 'https://live.staticflickr.com/7574/16249282061_0b4c4c558c_c.jpg',
 'https://live.staticflickr.com/4411/37197283130_68a7580fe5_c.jpg',
 'https://live.staticflickr.com/65535/49517907256_8ea57f8bc4_c.jpg',
 'https://live.sta

In [261]:
for url in enumerate(urls):
        print(url[1])
        

https://live.staticflickr.com/8077/8428940257_15f6a02bfd_c.jpg
https://live.staticflickr.com/8658/15752325014_612de967d8_c.jpg
https://live.staticflickr.com/3357/3333547544_814640b120_c.jpg
https://live.staticflickr.com/8622/16348796416_771690f621_c.jpg
https://live.staticflickr.com/8/10560929_d57c09863b_z.jpg
https://live.staticflickr.com/6/10560925_3660939718_c.jpg
https://live.staticflickr.com/5251/5398039808_4a34434460_c.jpg
https://live.staticflickr.com/6/10560928_0ebeb97562_z.jpg
https://live.staticflickr.com/127/321323177_3e057a4ef6_c.jpg
https://live.staticflickr.com/8785/18276105075_edef70fc09_c.jpg
https://live.staticflickr.com/5/10560927_e6a8465f45.jpg
https://live.staticflickr.com/4/4456745_cebf9bb9c5_n.jpg
https://live.staticflickr.com/7574/16249282061_0b4c4c558c_c.jpg
https://live.staticflickr.com/4411/37197283130_68a7580fe5_c.jpg
https://live.staticflickr.com/65535/49517907256_8ea57f8bc4_c.jpg
https://live.staticflickr.com/1510/25719431120_344a54f014_c.jpg
https://live.s

In [ ]:
for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join(ruta_destino, folder.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))

In [254]:
folder = 'cocido madrileño'
folder.split("_")[0]

'cocido madrileño'

In [259]:
folder = 'cocido madrileño fuerte'
folder.replace(" ", "_")

'cocido_madrileño_fuerte'

In [260]:
ruta_destino = '/home/dsc/tfm_data'
folder = 'cocido madrileño fuerte'
os.path.join(ruta_destino, folder.replace(" ", "_"),url[1].split("/")[-1])

'/home/dsc/tfm_data/cocido_madrileño_fuerte/30512631804_a6ce730650_c.jpg'

In [314]:
os.chdir("/home/dsc/TFM_food_recognition/image_urls")
file_name = 'cocido_urls.csv'
doc = pd.read_csv(file_name)  
doc.head(5)

,urls,download
0,https://live.staticflickr.com/8077/8428940257_...,False
1,https://live.staticflickr.com/8658/15752325014...,False
2,https://live.staticflickr.com/3357/3333547544_...,False
3,https://live.staticflickr.com/8622/16348796416...,False
4,https://live.staticflickr.com/8/10560929_d57c0...,False


In [307]:
doc[doc['download'] == False]

,urls,download
0,https://live.staticflickr.com/8077/8428940257_...,False
1,https://live.staticflickr.com/8658/15752325014...,False
2,https://live.staticflickr.com/3357/3333547544_...,False
3,https://live.staticflickr.com/8622/16348796416...,False
4,https://live.staticflickr.com/8/10560929_d57c0...,False
5,https://live.staticflickr.com/6/10560925_36609...,False
6,https://live.staticflickr.com/5251/5398039808_...,False
7,https://live.staticflickr.com/6/10560928_0ebeb...,False
8,https://live.staticflickr.com/127/321323177_3e...,False
9,https://live.staticflickr.com/8785/18276105075...,False


In [315]:
doc.iloc[0:10,1]=True

In [316]:
doc

,urls,download
0,https://live.staticflickr.com/8077/8428940257_...,True
1,https://live.staticflickr.com/8658/15752325014...,True
2,https://live.staticflickr.com/3357/3333547544_...,True
3,https://live.staticflickr.com/8622/16348796416...,True
4,https://live.staticflickr.com/8/10560929_d57c0...,True
5,https://live.staticflickr.com/6/10560925_36609...,True
6,https://live.staticflickr.com/5251/5398039808_...,True
7,https://live.staticflickr.com/6/10560928_0ebeb...,True
8,https://live.staticflickr.com/127/321323177_3e...,True
9,https://live.staticflickr.com/8785/18276105075...,True


In [317]:
new_urls = doc[doc['download'] == False]
new_urls

,urls,download
10,https://live.staticflickr.com/5/10560927_e6a84...,False
11,https://live.staticflickr.com/4/4456745_cebf9b...,False
12,https://live.staticflickr.com/7574/16249282061...,False
13,https://live.staticflickr.com/4411/37197283130...,False
14,https://live.staticflickr.com/65535/4951790725...,False
15,https://live.staticflickr.com/1510/25719431120...,False
16,https://live.staticflickr.com/181/417971459_41...,False
17,https://live.staticflickr.com/5755/23498640226...,False
18,https://live.staticflickr.com/7918/32863535128...,False
19,https://live.staticflickr.com/2196/2074438110_...,False


In [318]:
doc.loc[doc['download']==False, 'download'] = True

In [319]:
doc

,urls,download
0,https://live.staticflickr.com/8077/8428940257_...,True
1,https://live.staticflickr.com/8658/15752325014...,True
2,https://live.staticflickr.com/3357/3333547544_...,True
3,https://live.staticflickr.com/8622/16348796416...,True
4,https://live.staticflickr.com/8/10560929_d57c0...,True
5,https://live.staticflickr.com/6/10560925_36609...,True
6,https://live.staticflickr.com/5251/5398039808_...,True
7,https://live.staticflickr.com/6/10560928_0ebeb...,True
8,https://live.staticflickr.com/127/321323177_3e...,True
9,https://live.staticflickr.com/8785/18276105075...,True


In [256]:
ruta_destino = '/home/dsc/tfm_data'
os.path.join(ruta_destino, folder.split("_")[0],url[1].split("/")[-1])

'/home/dsc/tfm_data/cocido madrileño/30512631804_a6ce730650_c.jpg'

In [36]:
os.chdir('/home/dsc/TFM_food_recognition/image_urls')
os.getcwd()

'/home/dsc/TFM_food_recognition/image_urls'

In [7]:
!ls -l

total 1384
-rw-rw-r-- 1 dsc dsc  54346 feb 24 22:56  cats_and_dogs.ipynb
-rw-rw-r-- 1 dsc dsc  28300 mar  6 23:24 'causa limeña_urls.csv'
-rw-rw-r-- 1 dsc dsc  61537 mar  6 23:43  chaufa_urls.csv
-rw-rw-r-- 1 dsc dsc   6615 abr  8 23:08  cocido_urls.csv
-rw-rw-r-- 1 dsc dsc   1302 mar  6 18:23  croquetas.csv
-rw-rw-r-- 1 dsc dsc 101542 mar 14 21:43  croquetas_urls.csv
-rw-rw-r-- 1 dsc dsc  50321 mar  7 20:53 'ensaladilla rusa_urls.csv'
-rw-rw-r-- 1 dsc dsc 169815 mar 10 23:02  fabada_urls.csv
-rw-rw-r-- 1 dsc dsc  74201 abr  9 00:07  flickr_scrap.ipynb
-rw-rw-r-- 1 dsc dsc  58995 mar  7 21:06 'gambas al ajillo_urls.csv'
-rw-rw-r-- 1 dsc dsc  66907 mar  7 00:13  gazpacho_urls.csv
-rw-rw-r-- 1 dsc dsc  67093 mar  6 22:27  huancaina_urls.csv
-rw-rw-r-- 1 dsc dsc  62979 mar  7 23:02 'huevos rotos_urls.csv'
-rw-rw-r-- 1 dsc dsc 100608 mar 14 23:30  paella_urls.csv
-rw-rw-r-- 1 dsc dsc 220809 mar 16 23:38 'patatas bravas_urls.csv'
-rw-rw-r-- 1 dsc dsc  60636 mar  7 00:25  patatasbravas_urls.

In [43]:
urls_book

,Unnamed: 0,0
0,0,https://live.staticflickr.com/3025/2572581755_...
1,1,https://live.staticflickr.com/1234/543836157_b...
2,2,https://live.staticflickr.com/4913/45845152782...
3,3,https://live.staticflickr.com/3193/2573402624_...
4,4,https://live.staticflickr.com/4523/26880798669...
5,5,https://live.staticflickr.com/7006/6573762291_...
6,6,https://live.staticflickr.com/1368/543839969_a...
7,7,https://live.staticflickr.com/1254/543839975_2...
8,8,https://live.staticflickr.com/2749/4338186964_...
9,9,https://live.staticflickr.com/8500/8405657182_...


In [28]:
download_images(file_name = 'cocido_urls.csv', folder = 'cocido', 
               ruta_destino = '/home/dsc/tfm_data/')

Done downloading 1 of 10
Done downloading 2 of 10
Done downloading 3 of 10
Done downloading 4 of 10
Done downloading 5 of 10
Done downloading 6 of 10
Done downloading 7 of 10
Done downloading 8 of 10
Done downloading 9 of 10
Done downloading 10 of 10


Vamos a trabajar en esta parte del documento, cómo podemos recoger urls y ver si las tenemos (esas fotos) guardas en disco, si están, continuaremos buscando en flickr. Si no están, las guardamos y seguimos hasta llegar al máximo del contador: (para ello ejecutamos get_urls() y probamos con sus salidas)

In [21]:
urls[1]

'https://live.staticflickr.com/3366/3494696835_8db34cc247_c.jpg'

In [22]:
urls[1].split("/")[-1] # este va a ser el nombre definitivo de la foto. 

'3494696835_8db34cc247_c.jpg'

In [16]:
import requests
import os

In [25]:
os.path.join('/home/dsc/tfm_data/patatas bravas', urls[1].split("/")[-1])

'/home/dsc/tfm_data/patatas bravas/3494696835_8db34cc247_c.jpg'

In [24]:
if os.path.isfile(os.path.join('/home/dsc/tfm_data/patatas bravas', urls[1].split("/")[-1])):
        print('El archivo existe.');

El archivo existe.


In [23]:
for url in enumerate(urls):
    if os.path.isfile(os.path.join('/home/dsc/tfm_data/patatas bravas', url[1].split("/")[-1])):
        print('El archivo existe.');
    else:
        print('El archivo no existe')

El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo existe.
El archivo e

In [26]:
os.path.join('/home/dsc/tfm_data/', urls[1].split("/")[-1])

'/home/dsc/tfm_data/3494696835_8db34cc247_c.jpg'

In [54]:
if os.path.isfile(os.path.join('/home/dsc/tfm_data/patatas bravas', urls[1].split("/")[-1])):
    print('hola')

hola


In [50]:
if os.path.exists(os.path.join('/home/dsc/tfm_data/patatas bravas', urls[1].split("/")[-1])):
    print('hola')

hola


In [47]:
os.path.join('/home/dsc/tfm_data/patatas bravas', urls[0].split("/")[-1])


'/home/dsc/tfm_data/patatas bravas/48733297982_c17bca8a67_c.jpg'

In [45]:
prueba[0]

'https://live.staticflickr.com/5451/7392903470_b3b8aab810_c.jpg'

Podemos guardar el archivo de urls entero. No sobreescirbir sino cuando ejecutemos de nuevo la funcion de descargar imagenes, añadir las nuevas urls. Al final, cuando tengamos ya todas las imagenes, recogemoes de¡¡los nombres de las imagenes, hacemos un scan en el directorio, y solo nos quedamos con las urls que coinciden con nuestras imagenes. De esta forma tenemos la ruta de cada imagen que vamos a usar en el trabajo por si se pierden o se quieren descargar por cualquier otro motivo. 

Al punto anterior: mejor forma sería, cargo el archivo antiguo de urls en df, buscar la url en ese dataframe, si está, ignoro url, si no, la añado. Una vez tengamos un archivo de urls con cierto volumen, descargo las imagenes. 

In [8]:
import pandas as pd
pru = pd.read_csv('cocido_urls.csv')
pru2 = pd.read_csv('gazpacho_urls.csv', header=None)
pru.head()

,Unnamed: 0,0
0,0,https://live.staticflickr.com/3025/2572581755_...
1,1,https://live.staticflickr.com/1234/543836157_b...
2,2,https://live.staticflickr.com/4913/45845152782...
3,3,https://live.staticflickr.com/3193/2573402624_...
4,4,https://live.staticflickr.com/4523/26880798669...


In [12]:
pru.dtypes

0    float64
1     object
dtype: object

In [23]:
pru.shape

(10, 2)

In [13]:
pru.iloc[0, 1]

'https://live.staticflickr.com/3025/2572581755_a0137254f5_c.jpg'

Vamos a guardar esa url (string) en un objeto y vamos a buscarlo en el mismo dataframe para introducir en la función el asunto de las urls ya descargadas.

In [14]:
url_buscar = pru.iloc[0, 1]
url_buscar

'https://live.staticflickr.com/3025/2572581755_a0137254f5_c.jpg'

In [18]:
patron = url_buscar.split("/")[-1]
patron

'2572581755_a0137254f5_c.jpg'

In [20]:
if pru['0'].str.contains(patron).any():
    print('Url duplicada')

Url duplicada


In [10]:
pru2.head()

,0,1
0,0,https://live.staticflickr.com/3192/2829641312_...
1,1,https://live.staticflickr.com/3221/2677367663_...
2,2,https://live.staticflickr.com/4153/4841562401_...
3,3,https://live.staticflickr.com/4152/4840845965_...
4,4,https://live.staticflickr.com/7233/7203154546_...


In [24]:
pru2.shape

(1000, 2)

In [25]:
pru.append(pru2).shape

/home/dsc/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


(1010, 4)

In [25]:
pru.drop([0], axis=1, inplace=True)


KeyError: '[0] not found in axis'

In [26]:
pru.head()

,1
0,https://live.staticflickr.com/8077/8428940257_...
1,https://live.staticflickr.com/5755/23498640226...
2,https://live.staticflickr.com/7918/32863535128...
3,https://live.staticflickr.com/2196/2074438110_...
4,https://live.staticflickr.com/7826/46013936554...


In [28]:
pru.columns = ['url']
pru.head()

,url
0,https://live.staticflickr.com/8077/8428940257_...
1,https://live.staticflickr.com/5755/23498640226...
2,https://live.staticflickr.com/7918/32863535128...
3,https://live.staticflickr.com/2196/2074438110_...
4,https://live.staticflickr.com/7826/46013936554...


In [71]:
urls.head()

0    https://live.staticflickr.com/5451/7392903470_...
1    https://live.staticflickr.com/65535/4873329798...
2    https://live.staticflickr.com/2882/33876098296...
3    https://live.staticflickr.com/3366/3494696835_...
4    https://live.staticflickr.com/6088/6094691090_...
dtype: object

In [22]:
os.getcwd()

'/home/dsc/TFM_food_recognition'

In [77]:
archivo = pd.read_csv('tortilla de patata_urls.csv', header = None)

In [79]:
archivo.head()

,0,1
0,0,https://live.staticflickr.com/2129/2275422924_...
1,1,https://live.staticflickr.com/65535/4975099785...
2,2,https://live.staticflickr.com/7067/6778164752_...
3,3,https://live.staticflickr.com/65535/4898916235...
4,4,https://live.staticflickr.com/1186/707507320_5...


In [97]:
archivo.drop([0], axis=1, inplace=True)

In [98]:
archivo.head()

,1
0,https://live.staticflickr.com/2129/2275422924_...
1,https://live.staticflickr.com/65535/4975099785...
2,https://live.staticflickr.com/7067/6778164752_...
3,https://live.staticflickr.com/65535/4898916235...
4,https://live.staticflickr.com/1186/707507320_5...


In [80]:
urls.head()

0    https://live.staticflickr.com/5451/7392903470_...
1    https://live.staticflickr.com/65535/4873329798...
2    https://live.staticflickr.com/2882/33876098296...
3    https://live.staticflickr.com/3366/3494696835_...
4    https://live.staticflickr.com/6088/6094691090_...
dtype: object

In [81]:
pd.Series(urls)

0    https://live.staticflickr.com/5451/7392903470_...
1    https://live.staticflickr.com/65535/4873329798...
2    https://live.staticflickr.com/2882/33876098296...
3    https://live.staticflickr.com/3366/3494696835_...
4    https://live.staticflickr.com/6088/6094691090_...
5    https://live.staticflickr.com/65535/4971542127...
6    https://live.staticflickr.com/7237/7163937447_...
7    https://live.staticflickr.com/4894/45748942214...
8    https://live.staticflickr.com/7031/6453165539_...
9    https://live.staticflickr.com/7144/6453238363_...
dtype: object

In [86]:
df_urls = pd.DataFrame(urls)

In [90]:
df_urls

,0
0,https://live.staticflickr.com/5451/7392903470_...
1,https://live.staticflickr.com/65535/4873329798...
2,https://live.staticflickr.com/2882/33876098296...
3,https://live.staticflickr.com/3366/3494696835_...
4,https://live.staticflickr.com/6088/6094691090_...
5,https://live.staticflickr.com/65535/4971542127...
6,https://live.staticflickr.com/7237/7163937447_...
7,https://live.staticflickr.com/4894/45748942214...
8,https://live.staticflickr.com/7031/6453165539_...
9,https://live.staticflickr.com/7144/6453238363_...


In [104]:
archivo.shape

(100, 1)

In [106]:
df_urls.shape

(10, 1)

In [110]:
archivo=pd.DataFrame(archivo)

<bound method DataFrame.info of                                                     1
0   https://live.staticflickr.com/2129/2275422924_...
1   https://live.staticflickr.com/65535/4975099785...
2   https://live.staticflickr.com/7067/6778164752_...
3   https://live.staticflickr.com/65535/4898916235...
4   https://live.staticflickr.com/1186/707507320_5...
5   https://live.staticflickr.com/1543/25892955425...
6   https://live.staticflickr.com/2406/2198328634_...
7   https://live.staticflickr.com/3172/3118123794_...
8   https://live.staticflickr.com/2025/2275365168_...
9   https://live.staticflickr.com/7180/6794679794_...
10  https://live.staticflickr.com/2356/2275422906_...
11  https://live.staticflickr.com/5484/9193665856_...
12  https://live.staticflickr.com/8331/8116561402_...
13  https://live.staticflickr.com/8336/8116563582_...
14  https://live.staticflickr.com/680/22507967482_...
15  https://live.staticflickr.com/2762/4303688690_...
16  https://live.staticflickr.com/5443/7409362582_

In [5]:
def get_urls(image_tag, max):
    
    photos = get_photos(image_tag)
    counter=0
    urls=[]

    for photo in photos:
        if counter < max:
            counter += 1
            print("Fetching url for image number {}".format(counter))
            try:
                url = get_url(photo)  # get preffered size url
                #print(url)
                urls.append(url)
            except:    
                 print("Url for image number {} could not be fetched".format(count))
            # if no url for the desired sizes then try with the next photo
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls), max))
            break
    print("Writing out the urls in the current directory")
    urls = pd.Series(urls)
    urls.to_csv(image_tag + '_urls.csv')
    print('Listo!')
    

    return urls

In [9]:
import requests
import os

def download_images(file_name, image_tag):
    urls = []
    doc = pd.read_csv(file_name, header=None)
    
    for row in doc.iterrows():
        if row[1][1].startswith('https'):
            #print('yuhuuuu')
            urls.append(row[1][1])
    
    if not os.path.isdir(os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0])):
        os.mkdir(os.path.join('/home/dsc/tfm_data/', image_tag.split("_")[0]))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join('/home/dsc/tfm_data/',image_tag.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))

In [7]:
import requests
import os

def download_images(file_name, folder, ruta_destino):
    urls = []
    doc = pd.read_csv(file_name, header=None)
    
    for row in doc.iterrows():
        if row[1][1].startswith('https'):
            #print('yuhuuuu')
            urls.append(row[1][1])
    
    if not os.path.isdir(os.path.join(ruta_destino, folder.split("_")[0])):
        os.mkdir(os.path.join(ruta_destino, folder.split("_")[0]))
    
    
    for url in enumerate(urls):
        resp=requests.get(url[1], stream=True)
        path_to_write = os.path.join(ruta_destino, folder.split("_")[0],url[1].split("/")[-1])
        outfile=open(path_to_write,'wb')
        outfile.write(resp.content)
        outfile.close()
        print("Done downloading {} of {}".format(url[0]+1,len(urls)))